In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import io
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pdb

BATCH_SIZE = 32
VOCAB_SIZE = 50000

### Helper functions for each step in the pipeline

In [3]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    print(n, d)
    i = 0
    for line in fin:
        if i == VOCAB_SIZE:
            break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
        i += 1
    return data

In [4]:
from collections import Counter
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = 50000

def build_vocab():
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    word_vectors = pkl.load(open("fasttext_word_vectors.p", "rb"))
    id2token = list(word_vectors.keys())
    token2id = dict(zip(word_vectors, range(2,2+len(word_vectors)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return word_vectors, token2id, id2token

In [5]:
def convert_labels_to_integers(data_label):
    for i in range(len(data_label)):
        if data_label[i] == "contradiction":
            data_label[i] = 0
        elif data_label[i] == "entailment":
            data_label[i] = 1
        elif data_label[i] == "neutral":
            data_label[i] = 2
    return data_label

In [6]:
def verify_order(sent1_data, sent2_data, data_label):
    i = random.randint(1, len(sent1_data))
    print(sent1_data[i])
    print(sent2_data[i])
    print(data_label[i])

In [7]:
# Word tokenize each entry in a list of sentences
def tokenize(sentence_list):
    return [word_tokenize(sentence_list[i]) for i in range(len(sentence_list))]

In [8]:
# "one-hot encode": convert each token to id in vocabulary vector (token2id)
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data 

### Creating vocabulary & embedding matrix from FastText vectors

In [9]:
word_vectors, token2id, id2token = build_vocab()

In [10]:
_weights = np.array(list(word_vectors.values()))
pad_vec = np.zeros((1, 300))
unk_vec = np.random.randn(1, 300) * 0.01
pad_unk_vecs = np.vstack((pad_vec, unk_vec))
_WEIGHTS = np.vstack((pad_unk_vecs, _weights))
_WEIGHTS.shape

(50002, 300)

### Function to pre-process data for TwoSentenceModel
#### Shuffle, word tokenize, one-hot index into vocabulary

In [11]:
def text_pipeline(sent1s, sent2s, labels, verify=True):
    labels = convert_labels_to_integers(labels)
    seed = random.randint(1, 100)
    print("Random seed for shuffling: {}".format(seed))
    random.Random(seed).shuffle(sent1s)
    random.Random(seed).shuffle(sent2s)
    random.Random(seed).shuffle(labels)
    
    print("\nVerifying that the data and label match after shuffling")
    if verify:
        verify_order(sent1s, sent2s, labels)
        verify_order(sent1s, sent2s, labels)
          
    print("\nTokenizing sentence 1 list...")    
    sent1s_tokenized = tokenize(sent1s)
    print("done!")
    print("\nTokenizing sentence 2 list... ")  
    sent2s_tokenized = tokenize(sent2s)
    print("done!")
    
    print("\nOne-hot encoding words for sentence 1 list...")  
    sent1s_indices = token2index_dataset(sent1s_tokenized)
    print("done!")
    print("\nOne-hot encoding words for sentence 2 list...")  
    sent2s_indices = token2index_dataset(sent2s_tokenized)
    print("done!")
    
    return (sent1s_indices, sent2s_indices, labels)

### DataLoader

In [12]:
# MAX_SENTENCE_LENGTH = 30

import numpy as np
import torch
from torch.utils.data import Dataset

class TwoSentencesDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    """
    
    def __init__(self, sent1_data_list, sent2_data_list, target_list):
        """
        @param sent1_data_list: list of sentence1's (index matches sentence2's and target_list below)
        @param sent2_data_list: list of sentence2's
        @param target_list: list of correct labels

        """
        self.sent1_data_list = sent1_data_list
        self.sent2_data_list = sent2_data_list
        self.target_list = target_list
        assert (len(self.sent1_data_list) == len(self.target_list) and len(self.sent2_data_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_data_list)
        
    def __getitem__(self, key):
        ###
        ### Returns [[sentence, 1, tokens], [sentence, 2, tokens]]
        ###
        """
        Triggered when you call dataset[i]
        """
        sent1_tokens_idx = self.sent1_data_list[key][:MAX_SENTENCE_LENGTH]
        sent2_tokens_idx = self.sent2_data_list[key][:MAX_SENTENCE_LENGTH]
        combined_tokens_idx = [sent1_tokens_idx, sent2_tokens_idx]
        label = self.target_list[key]
        return [combined_tokens_idx, len(sent1_tokens_idx), len(sent2_tokens_idx), label]

def twosentences_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    sent1_data_list = []
    sent2_data_list = []
    sent1_length_list = []
    sent2_length_list = []
    label_list = []
    combined_data_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[3])
        sent1_length_list.append(datum[1])
        sent2_length_list.append(datum[2])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        combined_data_list.append([padded_vec_1, padded_vec_2])
    return [torch.from_numpy(np.array(combined_data_list)), 
            torch.LongTensor(sent1_length_list), torch.LongTensor(sent2_length_list), torch.LongTensor(label_list)]

### Train Dataset creation

In [13]:
snli_train = pd.read_csv('snli_train.tsv', sep='\t')
TRAIN_SIZE = 100000

sent1_data = list(snli_train["sentence1"])[:TRAIN_SIZE]
sent2_data = list(snli_train["sentence2"])[:TRAIN_SIZE]
data_label = list(snli_train["label"])[:TRAIN_SIZE]
print("Size of training data: {}".format(len(sent1_data)))

Size of training data: 100000


In [14]:
sent1_train_indices, sent2_train_indices, train_label = text_pipeline(sent1_data, sent2_data, data_label)
train_dataset = TwoSentencesDataset(sent1_train_indices, sent2_train_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )
print("Finished creating train_loader.")

Random seed for shuffling: 41

Verifying that the data and label match after shuffling
A man in a blue cap with a blue sweatshirt is talking on a microphone while a man without a shirt smiles .
The man is wearing a blue cap
1
Woman standing on a small mat with her mouth wide open .
A female has her mouth open and standing .
1

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!
Finished creating train_loader.


### Val Dataset creation

In [15]:
snli_val = pd.read_csv('snli_val.tsv', sep='\t')
sent1_val = list(snli_val["sentence1"])
sent2_val = list(snli_val["sentence2"])
val_label = list(snli_val["label"])
print("Size of val data: {}".format(len(sent1_val)))

Size of val data: 1000


In [16]:
sent1_val_indices, sent2_val_indices, val_label = text_pipeline(sent1_val, sent2_val, val_label)
val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

Random seed for shuffling: 57

Verifying that the data and label match after shuffling
A group of constructions workers lifting a bathtub .
Tall humans working
2
A horse and rider on a steeplechase course .
Three horses are playing in a field together .
0

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!


In [17]:
pd.Series([len(x) for x in snli_train['sentence1']]).describe()['75%']
MAX_SENTENCE_LENGTH = 40

### CNN Model

In [25]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(_WEIGHTS))
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool1d(30)
        self.linear1 = nn.Linear(2*hidden_size, 100)
        self.linear2 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, sent1_lengths, sent2_lengths):
        
        batch_size = x.size()[0]
        seq_len = x.size()[2]
        
        sent1s = torch.tensor(x[:, 0, :]).cuda()
        sent2s = torch.tensor(x[:, 1, :]).cuda()
        ordered_sents = torch.cat([sent1s, sent2s], dim=0).cuda()

        embed = self.embedding(ordered_sents)
        hidden = self.conv1(embed.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))

        hidden = self.conv2(hidden.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))
        hidden = self.maxpool(hidden.transpose(1, 2)).transpose(1, 2).squeeze(dim=1)
        
        hidden_sent1s = hidden[0:batch_size, :]
        hidden_sent2s = hidden[batch_size:, :]     
        
        linear1 = self.linear1(torch.cat([hidden_sent1s, hidden_sent2s], dim=1))
        linear1 = F.relu(linear1.contiguous().view(-1, linear1.size(-1))).view(linear1.shape)
        logits = self.linear2(linear1)
        return logits

In [19]:
# Function for testing the model
def test_model(loader, model):
    """
    Helper function that tests the model's performance on a dataset
    """
    correct = 0
    total = 0
    model.eval()
    for (data, sent1_lengths, sent2_lengths, labels) in loader:
        data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
        outputs = F.softmax(model(data_batch, sent1_length_batch, sent2_length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        labels = labels.cuda()
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def train_model(model, lr = 0.001, num_epochs = 7, criterion = nn.CrossEntropyLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

    for epoch in range(num_epochs):
        for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(data_batch, sent1_length_batch, sent2_length_batch)
            loss = criterion(outputs, label_batch)
            
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc))
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), loss))

In [26]:
model = CNN(emb_size = 300, hidden_size=300, num_layers=1, num_classes=3).cuda()
train_model(model)

Epoch: [1/7], Step: [101/3125], Validation Acc: 42.3
Epoch: [1/7], Step: [101/3125], Training Loss: 1.095030426979065
Epoch: [1/7], Step: [201/3125], Validation Acc: 46.0
Epoch: [1/7], Step: [201/3125], Training Loss: 1.001964807510376
Epoch: [1/7], Step: [301/3125], Validation Acc: 49.2
Epoch: [1/7], Step: [301/3125], Training Loss: 1.0298625230789185
Epoch: [1/7], Step: [401/3125], Validation Acc: 52.1
Epoch: [1/7], Step: [401/3125], Training Loss: 1.040648341178894
Epoch: [1/7], Step: [501/3125], Validation Acc: 56.3
Epoch: [1/7], Step: [501/3125], Training Loss: 0.9295246601104736
Epoch: [1/7], Step: [601/3125], Validation Acc: 51.0
Epoch: [1/7], Step: [601/3125], Training Loss: 1.2201602458953857
Epoch: [1/7], Step: [701/3125], Validation Acc: 54.3
Epoch: [1/7], Step: [701/3125], Training Loss: 0.8141111731529236
Epoch: [1/7], Step: [801/3125], Validation Acc: 58.9
Epoch: [1/7], Step: [801/3125], Training Loss: 0.8244349360466003
Epoch: [1/7], Step: [901/3125], Validation Acc: 59.

Epoch: [3/7], Step: [801/3125], Validation Acc: 63.8
Epoch: [3/7], Step: [801/3125], Training Loss: 0.5769374370574951
Epoch: [3/7], Step: [901/3125], Validation Acc: 64.2
Epoch: [3/7], Step: [901/3125], Training Loss: 0.7524006366729736
Epoch: [3/7], Step: [1001/3125], Validation Acc: 65.7
Epoch: [3/7], Step: [1001/3125], Training Loss: 0.9081165790557861
Epoch: [3/7], Step: [1101/3125], Validation Acc: 65.8
Epoch: [3/7], Step: [1101/3125], Training Loss: 0.6861642003059387
Epoch: [3/7], Step: [1201/3125], Validation Acc: 64.3
Epoch: [3/7], Step: [1201/3125], Training Loss: 0.9915214776992798
Epoch: [3/7], Step: [1301/3125], Validation Acc: 64.9
Epoch: [3/7], Step: [1301/3125], Training Loss: 0.7338667511940002
Epoch: [3/7], Step: [1401/3125], Validation Acc: 63.5
Epoch: [3/7], Step: [1401/3125], Training Loss: 1.0022474527359009
Epoch: [3/7], Step: [1501/3125], Validation Acc: 62.7
Epoch: [3/7], Step: [1501/3125], Training Loss: 0.6719067692756653
Epoch: [3/7], Step: [1601/3125], Val

Epoch: [5/7], Step: [1501/3125], Validation Acc: 64.2
Epoch: [5/7], Step: [1501/3125], Training Loss: 0.6389260292053223
Epoch: [5/7], Step: [1601/3125], Validation Acc: 65.8
Epoch: [5/7], Step: [1601/3125], Training Loss: 0.724966287612915
Epoch: [5/7], Step: [1701/3125], Validation Acc: 65.5
Epoch: [5/7], Step: [1701/3125], Training Loss: 0.6070282459259033
Epoch: [5/7], Step: [1801/3125], Validation Acc: 66.4
Epoch: [5/7], Step: [1801/3125], Training Loss: 0.710770845413208
Epoch: [5/7], Step: [1901/3125], Validation Acc: 64.3
Epoch: [5/7], Step: [1901/3125], Training Loss: 0.6709005832672119
Epoch: [5/7], Step: [2001/3125], Validation Acc: 65.8
Epoch: [5/7], Step: [2001/3125], Training Loss: 0.46004918217658997
Epoch: [5/7], Step: [2101/3125], Validation Acc: 63.5
Epoch: [5/7], Step: [2101/3125], Training Loss: 0.5361698269844055
Epoch: [5/7], Step: [2201/3125], Validation Acc: 63.5
Epoch: [5/7], Step: [2201/3125], Training Loss: 0.46074366569519043
Epoch: [5/7], Step: [2301/3125],

Epoch: [7/7], Step: [2101/3125], Validation Acc: 64.2
Epoch: [7/7], Step: [2101/3125], Training Loss: 0.42051997780799866
Epoch: [7/7], Step: [2201/3125], Validation Acc: 65.5
Epoch: [7/7], Step: [2201/3125], Training Loss: 0.41466695070266724
Epoch: [7/7], Step: [2301/3125], Validation Acc: 65.4
Epoch: [7/7], Step: [2301/3125], Training Loss: 0.5397405028343201
Epoch: [7/7], Step: [2401/3125], Validation Acc: 66.5
Epoch: [7/7], Step: [2401/3125], Training Loss: 0.6666191220283508
Epoch: [7/7], Step: [2501/3125], Validation Acc: 64.8
Epoch: [7/7], Step: [2501/3125], Training Loss: 0.6094503402709961
Epoch: [7/7], Step: [2601/3125], Validation Acc: 65.2
Epoch: [7/7], Step: [2601/3125], Training Loss: 0.33728328347206116
Epoch: [7/7], Step: [2701/3125], Validation Acc: 64.2
Epoch: [7/7], Step: [2701/3125], Training Loss: 0.5831938982009888
Epoch: [7/7], Step: [2801/3125], Validation Acc: 66.4
Epoch: [7/7], Step: [2801/3125], Training Loss: 0.5321475863456726
Epoch: [7/7], Step: [2901/312

In [ ]:
###
### Ideas
###

# Dropout layers --> prob 0.5
# weight decay. 

#rnn --> layer normalize

# CNN masking
# do not backpropagate
# after conv, cresate tensor masked not update.
# right after regular linear layer
# 
# set all elements until padding to 1